# Batch Gradient Descent

I'm going to implement Batch Gradient Descent with early stopping for Softmax Regression without using Scikit-Learn. I'll use to Iris data set from Scikit-learn to do this.

 - [Training and testing data](#Training-and-testing-data)
 - [Softmax regression training with early stopping](#Softmax-regression-training-with-early-stopping)
   - [Softmax function](#Softmax-function)
   - [Mean squared error](#Mean-squared-error)
 - [Early stopping](#Early-stopping)
 - [Predictions](#Predictions)

In [1]:
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()
print(iris['DESCR'])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

## Training and testing data

In [2]:
X = iris.data[:, (2,3)]
y = iris.target.reshape(-1,1)

In [3]:
def make_train_and_test_data(X, ratio):
    shuffled_index = np.random.permutation(X.shape[0])
    set_sample_size = int(len(X) * ratio)
    X_train = X[:set_sample_size]
    y_train = y[:set_sample_size]
    X_val = X[set_sample_size:]
    y_val = y[set_sample_size:]
    return X_train, y_train, X_val, y_val

In [4]:
X_train, y_train, X_val, y_val = make_train_and_test_data(X, 0.7)

In [5]:
X_train_b = np.c_[np.ones((X_train.shape[0], 1)), X_train]

In [6]:
eta = 0.001
n_iterations = 10000
theta = np.random.randn(X_train_b.shape[1], 1)
m = y_train.shape[0]

## Softmax regression training with early stopping

In [7]:
minimum_val_error = float('inf')
best_iteration = None
theta_arr = []

### Softmax function

In [8]:
def prob_softmax(score):
    return np.exp(score)/sum(np.exp(score))

### Mean squared error

In [9]:
def mean_squared_error(y_val, y_val_predict): 
    return np.sum((y_val - y_val_predict)**2)/len(y_val)

In [10]:
from time import time
t0 = time()
for iteration in range(n_iterations):
    prob_k = prob_softmax(X_train_b.dot(theta))
    gradients = 1/m * X_train_b.T.dot(prob_k - y_train)
    theta = theta - eta * gradients
    theta_arr.append(theta)
    y_predict = X_train_b.dot(theta)
    val_error = mean_squared_error(y_train, y_predict)
    if iteration % 500 == 0:
        print(iteration, val_error)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_iteration = iteration
print(f'Time elapsed: {time() - t0:.3f} seconds')

Time elapsed: 0.926 seconds


In [11]:
print(f'MSE of miminum value: {minimum_val_error:.6f}')
print(f'Index of best value: {best_iteration}')

MSE of miminum value: 0.025255
Index of best value: 767


In [12]:
best_theta = theta_arr[best_iteration]
print(best_theta)

[[-0.34405583]
 [ 0.08647061]
 [ 0.7542294 ]]


## Predictions

In [35]:
def predict_class(arr):
    X_new = np.array(arr)
    X_new_b = np.c_[np.ones((len(X_new), 1)), X_new]
    y_val = X_new_b.dot(best_theta)
    y_prob = prob_softmax(y_val)
    ind = np.unravel_index(np.argmax(y_prob, axis=None), y_prob.shape)
    return y_prob[ind]

In [36]:
predict_class(X_val)

0.03273145980964013